In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import ray

In [2]:
ray.init()

2020-01-20 10:46:26,346	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-01-20 10:46:26,372	INFO resource_spec.py:216 -- Starting Ray with 3.66 GiB memory available for workers and up to 1.84 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-01-20 10:46:26,696	WARNING services.py:1009 -- Failed to start the reporter. The reporter requires 'pip install psutil'.


{'node_ip_address': '172.17.0.2',
 'redis_address': '172.17.0.2:64550',
 'object_store_address': '/tmp/ray/session_2020-01-20_10-46-26_331998_1404/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-01-20_10-46-26_331998_1404/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-01-20_10-46-26_331998_1404'}

In [54]:
from functools import partial

DATA_DICT_NAME = "_generated_data"

class DGPGeneratorMethodWrapper():
    def __init__(self, generated_field, required_fields, func):
        self.generated_field = generated_field
        self.required_fields = required_fields
        self.func = func

    def __call__(self, *args, **kwargs):
        data_dict = getattr(args[0], DATA_DICT_NAME)

        required_field_vals = {
            k:data_dict[k]
            for k in self.required_fields
            if k in data_dict
        }
        
        if len(required_field_vals) != len(self.required_fields):
            raise Exception("Missing required value!")

        val = self.func(args[0], required_field_vals, *args[1:], **kwargs)
        data_dict[self.generated_field] = val
        
        return val

    def __get__(self, instance, owner):
        return partial(self.__call__, instance)

def dgp_generator_method(generated_field, required_fields):
    return partial(DGPGeneratorMethodWrapper, generated_field, required_fields)
        
class Test:
    def __init__(self):
        setattr(self, DATA_DICT_NAME, {})
        
    @dgp_generator_method("a", [])
    def test_method(self, required_field_vals):
        return 1
    
    @dgp_generator_method("b", ["a", "c"])
    def test_method_2(self, required_field_vals):
        return required_field_vals["a"] + 1
    
t = Test()

In [55]:
t.test_method()
t.test_method_2()

Exception: Missing required value!

In [53]:
t._generated_data

{'a': 1, 'b': 2}

In [8]:
l = ["a", "b"]
d = {
    "a": 1
}


SyntaxError: invalid syntax (<ipython-input-8-0ad92443e06c>, line 5)